# Table of Contents
* [GET User data](#chapter1)
* [Get Jump Page Clicks Number](#chapter2)
* [Update Segment](#chapter3)
* [Update Post Data](#chapter4)
* [Update Optout](#chapter5)
* [Update apikey](#chapter6)
* [Update Flow](#chapter7)
* [Update Creative Date](#chapter8)
* [Update Jobs](#chapter9)
* [Update Offers](#chapter10)

In [1]:
import pandas as pd
import glob
import os
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import filepath 
import infrastructure
import pygsheets 
import requests
import smartsheet
from requests.structures import CaseInsensitiveDict
from os import listdir
from os.path import isfile, join
import datetime as dt
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
current_date = now.strftime("%y%m%d")
print("Last update time:", current_time)
import json
import time

Last update time: 07/29/24 12:52:13


In [2]:
start_time = time.time()

In [3]:
filepath.smartsheet_folder

'D:\\Users\\lilig\\Desktop\\SMS-NewData\\smartsheet'

In [4]:
offer_sms = infrastructure.get_smartsheet('offers_sms')
test_sms = infrastructure.get_smartsheet('test_sms')

In [5]:
## Enter the data foler you place output the data file 
output_filepath = filepath.output_folder
## Enter the data foler you place store the input data 
input_filepath = filepath.input_folder
Segment_filepath = filepath.output_folder+'Segment/'
smsreport_filepath = filepath.output_folder+'SMS_Report/'
smsapikey_filepath = filepath.output_folder+'SMS_Apikey/'

In [6]:
offer_sms

,Hitpath Offer ID,Email Hitpath Offer ID,Scheduling Name,Offer Name,Status,Status Changed Date,Delayed Reporting,RX Rep,Advertiser,RX Biz Dev Rep,...,Budget Status,Budget,Cap,Portal Acct Info,Compliance Rules,Live Pub IDs,Paused Pubs,Special Pub Payouts,Pitch Approval,Single/Multiple Pitch
0,NaN,NaN,NaN,Charlie Financial [D] [N] [S] [SMS] - SMS,Pitch,07/29/24,NaN,Mary Beth Howe,Cx3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single
1,NaN,NaN,NaN,FastLoanHub.com,Pitch Denied,07/24/24,NaN,Surya Hemanth,Dot818,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single
2,NaN,NaN,Offers Approved/Declined ⬇️ drag offers here,NaN,-,NaN,NaN,NaN,Offers Ready to be Sorted,NaN,...,NaN,NaN,NaN,Offers Approved/Declined ⬇️,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,3 West Media,-,NaN,NaN,Sarah Bowe,3 West Media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13152.0,12980.0,13152-Credit Card-3 West Media-Indigo Masterca...,Indigo Mastercard CPL v3 (non-rewards),Set Up: Pending Custom Content,03/25/24,NaN,Sarah Bowe,3 West Media,NaN,...,Non Budgeted,NaN,NaN,3 West Media(ID:28528) - Haley - SMS,NaN,NaN,NaN,NaN,NaN,Single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:

offer_sms1 = offer_sms[offer_sms['Hitpath Offer ID'].isna()== False]
#offer_sms1['LC Offer'] =  offer_sms1['Hitpath Offer ID'].astype('str').str.split(".",expand = True)[0] + '_'+offer_sms1['Abbreviation'].str.split('_',expand = True)[1] 
#offer_sms1 = offer_sms1[offer_sms1['LC Offer'].isna()== False]

In [8]:
#offer_sms1['LC Offer'].to_csv('LC_offer.csv')

# GET User data <a class="anchor" id="chapter1"></a>

In [9]:
"""
# If you want to get the jump page click data from a specific date range 
start_date = '2023-05-01'
end_date = '2023-08-22'
# start_date  
date_range = pd.date_range(start=start_date, end=end_date)

url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=person&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
with open(output_filepath + 'User_information'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 

 """

'\n# If you want to get the jump page click data from a specific date range \nstart_date = \'2023-05-01\'\nend_date = \'2023-08-22\'\n# start_date  \ndate_range = pd.date_range(start=start_date, end=end_date)\n\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=person&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"\nresp = requests.get(url)\nwith open(output_filepath + \'User_information\'+current_date+\'.csv\', \'w+\') as f:\n    f.write(resp.text)\n# The code I downlodaed in the trinity, I want to combine it all in one files \n## Get the jump page clicks \n\n '

## Get Jump Page Clicks Number <a class="anchor" id="chapter2"></a>

In [10]:
""" 
# There's maximum 45 period windows to pull the  jump page clicks from API 
# Manual download the jump page click file from API
first_date_of_month = pd.to_datetime('2023-10-01')
last_date_of_month = pd.to_datetime('2023-10-31')
first_date_of_month_url = first_date_of_month.strftime('%m/%d/%Y')
yesterday = last_date_of_month.strftime('%m/%d/%Y')
current_date = first_date_of_month.strftime("%y%m")
url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
time.sleep(10)
with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
"""

' \n# There\'s maximum 45 period windows to pull the  jump page clicks from API \n# Manual download the jump page click file from API\nfirst_date_of_month = pd.to_datetime(\'2023-10-01\')\nlast_date_of_month = pd.to_datetime(\'2023-10-31\')\nfirst_date_of_month_url = first_date_of_month.strftime(\'%m/%d/%Y\')\nyesterday = last_date_of_month.strftime(\'%m/%d/%Y\')\ncurrent_date = first_date_of_month.strftime("%y%m")\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"\nresp = requests.get(url)\ntime.sleep(10)\nwith open(output_filepath + \'Jump Page Click from API/jumppage_click\'+current_date+\'.csv\', \'w+\') as f:\n    f.write(resp.text)\n'

In [11]:
#import new mamba 
# get active pubid 
mamba = infrastructure.get_mamba()
mamba['PUBID'] = mamba['Dataset'].str.split('_',expand = True)[2] 
active_pubid =mamba['PUBID'].unique().tolist() 
active_pubid.remove('461227') 
active_pubid.remove('461500') 
active_pubid.remove('461653') 
active_pubid.remove('461871') 
active_pubid.remove('461916')
active_pubid.remove('461794')
active_pubid.remove('461920')
middle_index = len(active_pubid) // 2
first_part = active_pubid[:middle_index]
second_part = active_pubid[middle_index:]

active_pubid_list = ['461227','461500','461653','461871','461916','461794','461920',','.join(first_part),','.join(second_part)] 
# yesterday 
yesterday = date.today() - timedelta(days=1) 
first_date_of_month = yesterday.replace(day=1) 
yesterday = yesterday.strftime('%D') 
#first_date_of_month_url = first_date_of_month.strftime('%m/%d/%y') 
first_date_of_month_url = first_date_of_month.strftime('%D') 
current_date = first_date_of_month.strftime("%y%m") 

## get last month's first date and last date 
lastMonth_last_date = first_date_of_month - timedelta(days=1) 
lastMonth_first_date = lastMonth_last_date.replace(day=1) 
#first_date_of_last_month_url = lastMonth_first_date.strftime('%m/%d/%y') 
first_date_of_last_month_url = lastMonth_first_date.strftime('%D') 
#last_date_last_month_url = lastMonth_last_date.strftime('%m/%d/%y')
last_date_last_month_url = lastMonth_last_date.strftime('%D') 
lastmonth_date = lastMonth_first_date.strftime("%y%m") 
# for i in active_pubid_list: 
#     url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={i}" 
#     resp = requests.get(url) 
#     print(url) 
#     current_month = resp.text 
#     time.sleep(25) 
#     if len(i) >10: 
#         with open(output_filepath + 'Jump Page Click from API/jumppage_click_others'+'_'+current_date+'.csv', 'w+') as f: 
#             f.write(resp.text)       
#     else: 
#         with open(output_filepath + 'Jump Page Click from API/jumppage_click_'+i+'_'+current_date+'.csv', 'w+') as f: 
#             f.write(resp.text) 
""" 
for i in active_pubid_list: 
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={i}" 
    resp = requests.get(url) 
    print(url) 
    current_month = resp.text 
    time.sleep(50) 
    if len(i) > 10:
        suffix = 'firsthalf' if i == ','.join(first_part) else 'secondhalf'
        with open(output_filepath + f'Jump Page Click from API/jumppage_click_{suffix}_{current_date}.csv', 'w+') as f:
            f.write(resp.text)
    else:
        with open(output_filepath + f'Jump Page Click from API/jumppage_click_{i}_{current_date}.csv', 'w+') as f:
            f.write(resp.text)
"""
def download_file(url, filepath):
    retry_count = 5
    for _ in range(retry_count):
        resp = requests.get(url)
        if len(resp.content) > 1024:  # Check if the content is greater than 1KB
            with open(filepath, 'w+') as f:
                f.write(resp.text)
            print(f"Downloaded: {filepath}")
            return True
        time.sleep(100)  # Wait for 5 seconds before retrying
    print(f"Failed to download: {filepath}")
    return False

for i in active_pubid_list:
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={i}"
    print(url)
   
    if len(i) > 10:
        suffix = 'firsthalf' if i == ','.join(first_part) else 'secondhalf'
        filepath = output_filepath + f'Jump Page Click from API/jumppage_click_{suffix}_{current_date}.csv'
    else:
        filepath = output_filepath + f'Jump Page Click from API/jumppage_click_{i}_{current_date}.csv'
   
    download_file(url, filepath)
    time.sleep(100)
            
# if date.today().day < 4:
#     for i in active_pubid_list: 
#         url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_last_month_url}&end={last_date_last_month_url}&agents={i}" 
#         resp = requests.get(url)
#         print(url) 
#         time.sleep(20) 
#         if len(i) >10: 
#             with open(output_filepath + 'Jump Page Click from API/jumppage_click_others'+'_'+lastmonth_date+'.csv', 'w+') as f: 
#                 f.write(resp.text) 
#         else: 
#             with open(output_filepath + 'Jump Page Click from API/jumppage_click_'+i+"_"+lastmonth_date+'.csv', 'w+') as f: 
#                 f.write(resp.text+ '\n') 
                
""" 
# If you want to get the jump page click data from a specific date range 
start_date = '2023-05-25' 
end_date = '2023-05-31' 
# start_date date_range = pd.date_range(start=start_date, end=end_date) 
for date in date_range: 
    current_date_url = date.strftime('%m/%d/%Y') 
    current_date = date.strftime("%y%m%d") 
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}" 
    resp = requests.get(url) 
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f: 
        f.write(resp.text) 
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 
os.chdir(output_filepath+'Jump Page Click from trinity') 
all_files = [i for i in glob.glob('*.{}'.format('csv'))] 
#combine all files in the list 
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ]) 
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']] 
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']] 
####Exporting master revenue file#### 
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202211-20230516.csv", index=False, encoding='utf-8-sig')


os.chdir(output_filepath+'jumppageclicks202309-202310')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202309-202310.csv", index=False, encoding='utf-8-sig')

os.chdir(output_filepath+'jumppageclicks202312-202401')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202312-202401.csv", index=False, encoding='utf-8-sig')
""" 

D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4928\718798223.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mamba['PUBID'] = mamba['Dataset'].str.split('_',expand = True)[2]


http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=07/01/24&end=07/28/24&agents=461227
Downloaded: D:\Users\lilig\Desktop\SMS-NewData\Jump Page Click from API/jumppage_click_461227_2407.csv
http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=07/01/24&end=07/28/24&agents=461500
Downloaded: D:\Users\lilig\Desktop\SMS-NewData\Jump Page Click from API/jumppage_click_461500_2407.csv
http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=07/01/24&end=07/28/24&agents=461653
Downloaded: D:\Users\lilig\Desktop\SMS-NewData\Jump Page Click from API/jumppage_click_461653_2407.csv
http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=07/01/24&end=07/28/24&agents=461871
Failed to download: D:\Users\lilig\Desktop\SMS-NewData\Jump Page Click from API/jumppage_click_461871_2407.csv
http://reporting.rxmgtracker.com/api.php?key=7e6ae66

' \n# If you want to get the jump page click data from a specific date range \nstart_date = \'2023-05-25\' \nend_date = \'2023-05-31\' \n# start_date date_range = pd.date_range(start=start_date, end=end_date) \nfor date in date_range: \n    current_date_url = date.strftime(\'%m/%d/%Y\') \n    current_date = date.strftime("%y%m%d") \n    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}" \n    resp = requests.get(url) \n    with open(output_filepath + \'Jump Page Click from API/jumppage_click\'+current_date+\'.csv\', \'w+\') as f: \n        f.write(resp.text) \n# The code I downlodaed in the trinity, I want to combine it all in one files \n## Get the jump page clicks \nos.chdir(output_filepath+\'Jump Page Click from trinity\') \nall_files = [i for i in glob.glob(\'*.{}\'.format(\'csv\'))] \n#combine all files in the list \ncombined_jump_page_csv = pd.concat([pd.read_csv(

In [12]:
if date.today().day < 4:
    for i in active_pubid_list: 
        url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_last_month_url}&end={last_date_last_month_url}&agents={i}" 
        print(url) 
        if len(i) > 10:
            suffix = 'firsthalf' if i == ','.join(first_part) else 'secondhalf'
            filepath = output_filepath + f'Jump Page Click from API/jumppage_click_{suffix}_{lastmonth_date}.csv'
        else:
            filepath = output_filepath + f'Jump Page Click from API/jumppage_click_{i}_{lastmonth_date}.csv'
        download_file(url, filepath)
        time.sleep(50)

In [13]:
# os.chdir(output_filepath+'jumppageclicks202402-202403')
# all_files = [i for i in glob.glob('*.{}'.format('csv'))]
# #combine all files in the list
# combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
# combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
# combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
# ####Exporting master revenue file####
# combined_jump_page_csv.to_csv(output_filepath+"jumppageclicks202402-202403.csv", index=False, encoding='utf-8-sig')

In [14]:


# if date.today().day < 4:
#     for i in active_pubid_list: 
#         url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_last_month_url}&end={last_date_last_month_url}&agents={i}" 
#         resp = requests.get(url) 
#         print(url) 
#         time.sleep(50) 
#         if len(i) > 10:
#             suffix = 'firsthalf' if i == ','.join(first_part) else 'secondhalf'
#             with open(output_filepath + f'Jump Page Click from API/jumppage_click_{suffix}'+'_'+lastmonth_date+'.csv', 'w+') as f:
#                 f.write(resp.text)
#         else:
#             with open(output_filepath + f'Jump Page Click from API/jumppage_click_'+i+"_"+lastmonth_date+'.csv', 'w+') as f:
#                 f.write(resp.text+ '\n')


In [15]:
os.chdir(output_filepath+'Jump Page Click from API') 
all_files = [i for i in glob.glob('*.{}'.format('csv'))] 
#combine all files in the list 
#api_jump = pd.concat([pd.read_csv(f) for f in all_files ]) 
api_jump = pd.concat([pd.read_csv(f, encoding='ISO-8859-1') for f in all_files])
api_jump = api_jump.rename(columns = {'c1': 'subid_1','campaignid': 'campaign_id'}) 
api_jump['affiliate_id']= api_jump['subid_1'].astype(str).str.extract(r'(46\d{4})') 
api_jump = api_jump[['date','subid_1','campaign_id','affiliate_id','hitid']] 
####Exporting master revenue file#### 
jump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")
jump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")
jump_page12230124 = pd.read_csv(output_filepath+"jumppage_click202312-202401.csv")
jump_page240203=pd.read_csv(output_filepath+"jumppageclicks202402-202403.csv")
jump_page = pd.concat([jump_page_csv,api_jump,jump_page091023,jump_page12230124,jump_page240203]) 

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4928\1522374224.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  api_jump = pd.concat([pd.read_csv(f, encoding='ISO-8859-1') for f in all_files])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4928\1522374224.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  api_jump = pd.concat([pd.read_csv(f, encoding='ISO-8859-1') for f in all_files])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4928\1522374224.py:5: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  api_jump = pd.concat([pd.read_csv(f, encoding='ISO-8859-1') for f in all_files])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4928\1522374224.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  api_jump = pd.concat([pd.read_csv(f, encoding='ISO-8859-1') for f in all_fi

In [16]:

jump_page_clicks = jump_page.groupby(['affiliate_id','campaign_id','subid_1'])['hitid'].nunique().reset_index(name = 'Jump Page Clicks') 
affiliate_id = jump_page_clicks['affiliate_id'].astype(str).unique().tolist() 
jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)] 
jump_page_clicks['date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})') 
# convert the dateinfo to datetime format using pd.to_datetime() 
jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date'], format='%d%b%y', errors = 'coerce') 
#process with subid1 LC to TF if needed
def replace_second_part_with_TF(subid):
    parts = subid.split('_', 2)
    if len(parts) > 2:
        parts[1] = 'TF'
    return '_'.join(parts)
# Apply the transformation where 'affiliate_id' is '461871'
jump_page_clicks.loc[jump_page_clicks['affiliate_id'] == '461871', 'subid_1'] = jump_page_clicks.loc[jump_page_clicks['affiliate_id'] == '461871', 'subid_1'].apply(replace_second_part_with_TF)
jump_page_clicks.loc[jump_page_clicks['subid_1'].str.split('_').str[1] == 'TF', 'Jump Page Clicks'] = 0
jump_page_clicks.to_csv(output_filepath+ "jumppage_click.csv", index=False, encoding='utf-8-sig') 
duplicateRows = jump_page[jump_page['hitid'].duplicated()]
duplicateRows 

# if there's duplicates rows, please check!
"""


os.chdir(output_filepath+'Jump Page Click from API')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
api_jump = api_jump.rename(columns = {'c1': 'subid_1','campaignid': 'campaign_id'})
api_jump['subid_1'] = api_jump['subid_1'].fillna('').astype(str)
api_jump['affiliate_id']= api_jump['subid_1'].astype(str).str.extract(r'(46\d{4})')
api_jump = api_jump[['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
jump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")
jump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")
jump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])
jump_page_clicks = jump_page.groupby(['affiliate_id','campaign_id','subid_1'])['hitid'].nunique().reset_index(name = 'Jump Page Clicks')
affiliate_id = jump_page_clicks['affiliate_id'].astype(str).unique().tolist()
jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]
jump_page_clicks['date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
#added errors = coerce to passby format error
jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date'], format='%d%b%y', errors = 'coerce') 
#jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date']) 
#jump_page_clicks['date'] = jump_page_clicks['date'].dt.strftime('%d%b%y')
jump_page_clicks.to_csv(output_filepath+ "jumppage_click.csv", index=False, encoding='utf-8-sig')
duplicateRows = jump_page[jump_page['hitid'].duplicated()]
duplicateRows
# if there's duplicates rows, please check!
"""

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4928\3268183117.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]


'\n\n\nos.chdir(output_filepath+\'Jump Page Click from API\')\n\nall_files = [i for i in glob.glob(\'*.{}\'.format(\'csv\'))]\n#combine all files in the list\napi_jump = pd.concat([pd.read_csv(f) for f in all_files ])\napi_jump = api_jump.rename(columns = {\'c1\': \'subid_1\',\'campaignid\': \'campaign_id\'})\napi_jump[\'subid_1\'] = api_jump[\'subid_1\'].fillna(\'\').astype(str)\napi_jump[\'affiliate_id\']= api_jump[\'subid_1\'].astype(str).str.extract(r\'(46\\d{4})\')\napi_jump = api_jump[[\'date\',\'subid_1\',\'campaign_id\',\'affiliate_id\',\'hitid\']]\n####Exporting master revenue file####\njump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")\njump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")\njump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])\njump_page_clicks = jump_page.groupby([\'affiliate_id\',\'campaign_id\',\'subid_1\'])[\'hitid\'].nunique().reset_index(name = \'Jump Page Clicks\')\naffiliate_id = jump_

In [17]:
# check = jump_page_clicks[jump_page_clicks['affiliate_id'] == '461871']
# check

In [18]:
# total_clicks_affiliate_461871 = jump_page_clicks[jump_page_clicks['affiliate_id'] == '461871']['Jump Page Clicks'].sum()
# total_clicks_affiliate_461871

In [19]:
# total_jump_page_clicks = jump_page_clicks[jump_page_clicks['subid_1'].str.split('_').str[1] == 'TF']['Jump Page Clicks'].sum()
# total_jump_page_clicks

In [20]:
""""

#import new mamba 
# get active pubid 
mamba = infrastructure.get_mamba()
mamba['PUBID'] = mamba['Dataset'].str.split('_',expand = True)[2]
active_pubid =','.join(mamba['PUBID'].unique().tolist())
active_pubid += ',460001'
# yesterday
yesterday = date.today() - timedelta(days=1)
first_date_of_month = yesterday.replace(day=1)
yesterday = yesterday.strftime('%m/%d/%Y')
first_date_of_month_url = first_date_of_month.strftime('%m/%d/%Y')
current_date =  first_date_of_month.strftime("%y%m")

## get last month's first date and last date 
lastMonth_last_date = first_date_of_month - timedelta(days=1)
lastMonth_first_date = lastMonth_last_date.replace(day=1)
first_date_of_last_month_url = lastMonth_first_date.strftime('%m/%d/%Y')
last_date_last_month_url = lastMonth_last_date.strftime('%m/%d/%Y')
lastmonth_date =  lastMonth_first_date.strftime("%y%m")

url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
time.sleep(10)
with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
if date.today().day < 10:
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_last_month_url}&end={last_date_last_month_url}&agents={active_pubid}"
    resp = requests.get(url)
    time.sleep(10)
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+lastmonth_date+'.csv', 'w+') as f:
        f.write(resp.text)

# If you want to get the jump page click data from a specific date range 
start_date = '2023-09-01'
end_date = '2023-09-01'
# start_date  
date_range = pd.date_range(start=start_date, end=end_date)
for date in date_range:
    current_date_url = date.strftime('%m/%d/%Y')
    current_date = date.strftime("%y%m%d")
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}"
    resp = requests.get(url)
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
        f.write(resp.text)
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 

os.chdir(output_filepath+'Jump Page Click from trinity')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202211-20230516.csv", index=False, encoding='utf-8-sig')

"""

'"\n\n#import new mamba \n# get active pubid \nmamba = infrastructure.get_mamba()\nmamba[\'PUBID\'] = mamba[\'Dataset\'].str.split(\'_\',expand = True)[2]\nactive_pubid =\',\'.join(mamba[\'PUBID\'].unique().tolist())\nactive_pubid += \',460001\'\n# yesterday\nyesterday = date.today() - timedelta(days=1)\nfirst_date_of_month = yesterday.replace(day=1)\nyesterday = yesterday.strftime(\'%m/%d/%Y\')\nfirst_date_of_month_url = first_date_of_month.strftime(\'%m/%d/%Y\')\ncurrent_date =  first_date_of_month.strftime("%y%m")\n\n## get last month\'s first date and last date \nlastMonth_last_date = first_date_of_month - timedelta(days=1)\nlastMonth_first_date = lastMonth_last_date.replace(day=1)\nfirst_date_of_last_month_url = lastMonth_first_date.strftime(\'%m/%d/%Y\')\nlast_date_last_month_url = lastMonth_last_date.strftime(\'%m/%d/%Y\')\nlastmonth_date =  lastMonth_first_date.strftime("%y%m")\n\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&

In [21]:
print(url)

http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=07/01/24&end=07/28/24&agents=461452,461242,461247,461842,461843,461768,461039,461896,461933,461919,461944


In [22]:
"""

os.chdir(output_filepath+'Jump Page Click from API')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
api_jump = api_jump.rename(columns = {'c1': 'subid_1','campaignid': 'campaign_id'})
api_jump['subid_1'] = api_jump['subid_1'].fillna('').astype(str)
api_jump['affiliate_id']= api_jump['subid_1'].astype(str).str.extract(r'(46\d{4})')
api_jump = api_jump[['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
jump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")
jump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")
jump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])
jump_page_clicks = jump_page.groupby(['affiliate_id','campaign_id','subid_1'])['hitid'].nunique().reset_index(name = 'Jump Page Clicks')
affiliate_id = jump_page_clicks['affiliate_id'].astype(str).unique().tolist()
jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]
jump_page_clicks['date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
#added errors = coerce to passby format error
jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date'], format='%d%b%y', errors = 'coerce') 
#jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date']) 
#jump_page_clicks['date'] = jump_page_clicks['date'].dt.strftime('%d%b%y')
jump_page_clicks.to_csv(output_filepath+ "jumppage_click.csv", index=False, encoding='utf-8-sig')
duplicateRows = jump_page[jump_page['hitid'].duplicated()]
duplicateRows
# if there's duplicates rows, please check!
"""

'\n\nos.chdir(output_filepath+\'Jump Page Click from API\')\n\nall_files = [i for i in glob.glob(\'*.{}\'.format(\'csv\'))]\n#combine all files in the list\napi_jump = pd.concat([pd.read_csv(f) for f in all_files ])\napi_jump = api_jump.rename(columns = {\'c1\': \'subid_1\',\'campaignid\': \'campaign_id\'})\napi_jump[\'subid_1\'] = api_jump[\'subid_1\'].fillna(\'\').astype(str)\napi_jump[\'affiliate_id\']= api_jump[\'subid_1\'].astype(str).str.extract(r\'(46\\d{4})\')\napi_jump = api_jump[[\'date\',\'subid_1\',\'campaign_id\',\'affiliate_id\',\'hitid\']]\n####Exporting master revenue file####\njump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")\njump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")\njump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])\njump_page_clicks = jump_page.groupby([\'affiliate_id\',\'campaign_id\',\'subid_1\'])[\'hitid\'].nunique().reset_index(name = \'Jump Page Clicks\')\naffiliate_id = jump_pa

In [23]:
"""
jump_page_clicks['Date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
# added errors = coerce to bypass format errors
jump_page_clicks['Date'] = pd.to_datetime(jump_page_clicks['Date'], format='%d%b%y', errors = 'coerce') 
jump_page_clicks
"""

"\njump_page_clicks['Date'] = jump_page_clicks['subid_1'].str.extract(r'(\\d{1,2}[A-Za-z]{3}\\d{2})')\n# convert the dateinfo to datetime format using pd.to_datetime()\n# added errors = coerce to bypass format errors\njump_page_clicks['Date'] = pd.to_datetime(jump_page_clicks['Date'], format='%d%b%y', errors = 'coerce') \njump_page_clicks\n"

In [24]:
total_jump_page_clicks = jump_page_clicks[jump_page_clicks['subid_1'].str.split('_').str[1] == 'TF']['Jump Page Clicks'].sum()
total_jump_page_clicks

0

In [25]:

useful_click = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]['Jump Page Clicks'].sum()
useless_clicks = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False) == False]['Jump Page Clicks'].sum()
useful_click+useless_clicks -  len(jump_page) 
# the missing 502 clicks is from problematic subid_1 
# useless clicks is also from porblematic clicks 

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4928\322886987.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  useful_click = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]['Jump Page Clicks'].sum()
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4928\322886987.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  useless_clicks = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False) == False]['Jump Page Clicks'].sum()


-5957683

## Update Segment <a class="anchor" id="chapter3"></a>

In [26]:

from os import listdir
from os.path import isfile, join
import datetime as dt

mamba = infrastructure.get_mamba()
current_active_segment = mamba.loc[mamba['Date'] == pd.to_datetime(date.today()),'Segment '].unique().tolist()
secondary_segment =  mamba.loc[(mamba['Date'] == pd.to_datetime(date.today())) & (mamba['Segment '].str.contains('TMO|VZN',na = False)),'Segment '].unique().tolist()

                                                                 
segment_files = [f for f in listdir(Segment_filepath) if isfile(join(Segment_filepath, f)) & ('SMS_SC_SS_Segments' in f)]
output_segment = pd.DataFrame()
segment_files.sort()
for i in segment_files: 
    segment_report = pd.read_csv(Segment_filepath+i)
    segment_report['Date'] = dt.datetime.strptime((i.split('.csv')[0][-8:]), "%Y%m%d")
    updated_segment_report = segment_report.drop(['id','next_count'],axis = 1)
    output_segment = pd.concat([updated_segment_report,output_segment], axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
    print(output_segment)
""" 
most_recent_segment_date = max([dt.datetime.strptime((date.split('.csv')[0][-8:]), "%Y%m%d") for date in segment_files])
segment_year = most_recent_segment_date.year
segment_month = most_recent_segment_date.month
segement_day = most_recent_segment_date.day

most_recent_segment_file = "SMS_SC_SS_Segments{0}{1:0>2}{2:0>2}.csv".format(segment_year,segment_month,segement_day)

segment_report = pd.read_csv( Segment_filepath+ most_recent_segment_file)

segment_report['Date'] = pd.to_datetime(dt.datetime.today().strftime("%D"))
updated_segment_report = segment_report.drop(['id','next_count'],axis = 1)

legacy_segment = pd.read_csv(output_filepath+'SMS_SC_SS_Segments.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_segment.to_csv(input_filepath+'SMS_SC_SS_Segments_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
legacy_segment['Date'] = pd.to_datetime(legacy_segment['Date'])
min_date = updated_segment_report['Date'].min()
legacy_segment = legacy_segment[legacy_segment['Date']<min_date]
full_segment = [legacy_segment, updated_segment_report]
output_segment = pd.concat(full_segment, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_segment)
"""
## rename on 5/24
output_segment['name'] = output_segment['name'].str.replace('FLC_I.CC_OI.4P_30DC','FLC_I.CC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_SPK.CR_OI.4P_30DC','FLC_SPK.CR_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_SPK.SWP_OI.4P_30DC','MBC_SPK.SWP_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_AI.CC_OI.4P_30DC','FLC_AI.CC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_EDM.247L_OI.4P_30DC','FLC_EDM.247L_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_NPD.RTO_OI.4P_30DC','MBC_NPD.RTO_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_PN.FC_OI.4P_30DC','MBC_PN.FC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_WWM.YFA.2_OI.4P_30DC','FLC_WWM.YFA.2_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_ZM.PL.2_OI.4P_30DC','FLC_ZM.PL.2_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_SPK.SWP_OI.4P.VZN_30DC','FLC_SPK.SWP_30DC_VZN')
output_segment['name'] = output_segment['name'].str.replace('MBC_EDM.247L_OI.4P.TMO_30DC','MBC_EDM.247L_30DC_TMO')
output_segment['name'] = output_segment['name'].str.replace('MBC_WWM.YFA.2_OI.4P_30DC','MBC_WWM.YFA.2_30DC_TMO')
output_segment['Secondary Segment'] = False
output_segment['Active Segment'] = False
output_segment.loc[output_segment['name'].isin(secondary_segment),'Secondary Segment'] = True
output_segment.loc[output_segment['name'].isin(current_active_segment),'Active Segment'] = True

output_segment.to_csv(output_filepath+'SMS_SC_SS_Segments.csv', index = False)

D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


                             name channel  status  count  \
0                          461243     sms  active  11531   
1          526 rto-JULY22-CLK-hzb     sms  active   1283   
2                   ARM CR 460939     sms  active  62935   
3               ARM.CR Last Month     sms  active  26541   
4                   ARMSMPL461149     sms  active   8279   
..                            ...     ...     ...    ...   
304                RTO-JULY22-CLK     sms  active   7579   
305               test-2565371505     sms  active   4255   
306  viewrenttoownhomes URL check     sms  active   6386   
307                      ZP461235     sms  active  85116   
308                 ZP Last Month     sms  active   4110   

                last_counted       Date  
0    2023-05-01 00:06:08 EST 2023-05-17  
1    2023-05-17 06:18:34 EST 2023-05-17  
2    2023-05-01 19:42:33 EST 2023-05-17  
3    2023-04-29 06:55:26 EST 2023-05-17  
4    2023-04-29 04:34:56 EST 2023-05-17  
..                       ..

                              name channel  status  count  \
0                           461243     sms  active  11531   
1           526 rto-JULY22-CLK-hzb     sms  active    944   
2                    ARMSMPL461149     sms  active   8279   
3                ARMSYF Last Month     sms  active      0   
4                   ARMUSACF461148     sms  active   2690   
...                            ...     ...     ...    ...   
4137                RTO-JULY22-CLK     sms  active   7579   
4138               test-2565371505     sms  active   4255   
4139  viewrenttoownhomes URL check     sms  active   6386   
4140                      ZP461235     sms  active  85116   
4141                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-05-01 00:06:08 EST 2023-06-02  
1     2023-06-02 06:16:30 EST 2023-06-02  
2     2023-04-29 04:34:56 EST 2023-06-02  
3     2023-04-29 05:14:10 EST 2023-06-02  
4     2023-04-29 03:27:33 EST 2023-06-02  
...      

                              name channel  status  count  \
0           526 rto-JULY22-CLK-hzb     sms  active    942   
1                    ARMSMPL461149     sms  active   8279   
2                   ARMUSACF461148     sms  active   2690   
3                     B2CGC 461122     sms  active   7417   
4                B2USRA Last Month     sms  active     36   
...                            ...     ...     ...    ...   
7063                RTO-JULY22-CLK     sms  active   7579   
7064               test-2565371505     sms  active   4255   
7065  viewrenttoownhomes URL check     sms  active   6386   
7066                      ZP461235     sms  active  85116   
7067                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-06-20 09:36:10 EST 2023-06-20  
1     2023-04-29 04:34:56 EST 2023-06-20  
2     2023-04-29 03:27:33 EST 2023-06-20  
3     2023-05-01 13:44:37 EST 2023-06-20  
4     2023-04-29 07:33:25 EST 2023-06-20  
...      

                              name channel  status  count  \
0           526 rto-JULY22-CLK-hzb     sms  active    945   
1                    ARMSMPL461149     sms  active   8279   
2                   ARMUSACF461148     sms  active   2690   
3                     B2CGC 461122     sms  active   7417   
4                B2USRA Last Month     sms  active     36   
...                            ...     ...     ...    ...   
8816                RTO-JULY22-CLK     sms  active   7579   
8817               test-2565371505     sms  active   4255   
8818  viewrenttoownhomes URL check     sms  active   6386   
8819                      ZP461235     sms  active  85116   
8820                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-06-29 07:11:36 EST 2023-06-29  
1     2023-04-29 04:34:56 EST 2023-06-29  
2     2023-04-29 03:27:33 EST 2023-06-29  
3     2023-05-01 13:44:37 EST 2023-06-29  
4     2023-04-29 07:33:25 EST 2023-06-29  
...      

...

                               name channel  status  count  \
0            526 rto-JULY22-CLK-hzb     sms  active    907   
1                    A4F_I.A4F_15DC     sms  active      0   
2                     A4F_I.A4F_7DC     sms  active      0   
3          ALL HZB_RHD.CC_all codes     sms  active  32716   
4          Copy Copy MBC_PN.FC_30DC     sms  active   8626   
...                             ...     ...     ...    ...   
92151                RTO-JULY22-CLK     sms  active   7579   
92152               test-2565371505     sms  active   4255   
92153  viewrenttoownhomes URL check     sms  active   6386   
92154                      ZP461235     sms  active  85116   
92155                 ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-05-29 06:18:39 EST 2024-05-29  
1      2024-05-29 07:33:58 EST 2024-05-29  
2      2024-05-29 07:33:34 EST 2024-05-29  
3      2024-05-29 10:06:35 EST 2024-05-29  
4      2024-05-29 05:51:17 EST 2024

                               name channel  status  count  \
0            526 rto-JULY22-CLK-hzb     sms  active    904   
1                    A4F_I.A4F_15DC     sms  active      0   
2                     A4F_I.A4F_7DC     sms  active      0   
3          ALL HZB_RHD.CC_all codes     sms  active  31697   
4          Copy Copy MBC_PN.FC_30DC     sms  active   8233   
...                             ...     ...     ...    ...   
96619                RTO-JULY22-CLK     sms  active   7579   
96620               test-2565371505     sms  active   4255   
96621  viewrenttoownhomes URL check     sms  active   6386   
96622                      ZP461235     sms  active  85116   
96623                 ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-06-10 08:37:12 EST 2024-06-10  
1      2024-06-10 10:08:08 EST 2024-06-10  
2      2024-06-10 10:09:50 EST 2024-06-10  
3      2024-06-10 06:41:11 EST 2024-06-10  
4      2024-06-10 08:08:15 EST 2024

                                name channel  status  count  \
0             526 rto-JULY22-CLK-hzb     sms  active    895   
1                     A4F_I.A4F_15DC     sms  active      0   
2                      A4F_I.A4F_7DC     sms  active      0   
3           ALL HZB_RHD.CC_all codes     sms  active  30685   
4           Copy Copy MBC_PN.FC_30DC     sms  active   8446   
...                              ...     ...     ...    ...   
102323                RTO-JULY22-CLK     sms  active   7579   
102324               test-2565371505     sms  active   4255   
102325  viewrenttoownhomes URL check     sms  active   6386   
102326                      ZP461235     sms  active  85116   
102327                 ZP Last Month     sms  active   4110   

                   last_counted       Date  
0       2024-06-25 07:50:23 EST 2024-06-25  
1       2024-06-25 09:10:14 EST 2024-06-25  
2       2024-06-25 09:10:04 EST 2024-06-25  
3       2024-06-25 06:47:51 EST 2024-06-25  
4       2024-06-25

                                name channel  status  count  \
0             526 rto-JULY22-CLK-hzb     sms  active    532   
1                     A4F_I.A4F_15DC     sms  active      0   
2                      A4F_I.A4F_7DC     sms  active      0   
3           ALL HZB_RHD.CC_all codes     sms  active  27121   
4           Copy Copy MBC_PN.FC_30DC     sms  active   8154   
...                              ...     ...     ...    ...   
108245                RTO-JULY22-CLK     sms  active   7579   
108246               test-2565371505     sms  active   4255   
108247  viewrenttoownhomes URL check     sms  active   6386   
108248                      ZP461235     sms  active  85116   
108249                 ZP Last Month     sms  active   4110   

                   last_counted       Date  
0       2024-07-09 14:58:05 EST 2024-07-11  
1       2024-07-09 16:39:21 EST 2024-07-11  
2       2024-07-09 16:40:08 EST 2024-07-11  
3       2024-07-09 14:38:32 EST 2024-07-11  
4       2024-07-09

                                name channel  status  count  \
0             526 rto-JULY22-CLK-hzb     sms  active    524   
1                     A4F_I.A4F_15DC     sms  active      0   
2                      A4F_I.A4F_7DC     sms  active      0   
3           ALL HZB_RHD.CC_all codes     sms  active  26128   
4           Copy Copy MBC_PN.FC_30DC     sms  active   7614   
...                              ...     ...     ...    ...   
114341                RTO-JULY22-CLK     sms  active   7579   
114342               test-2565371505     sms  active   4255   
114343  viewrenttoownhomes URL check     sms  active   6386   
114344                      ZP461235     sms  active  85116   
114345                 ZP Last Month     sms  active   4110   

                   last_counted       Date  
0       2024-07-24 00:12:29 EST 2024-07-25  
1       2024-07-24 10:20:45 EST 2024-07-25  
2       2024-07-24 10:21:36 EST 2024-07-25  
3       2024-07-23 23:43:25 EST 2024-07-25  
4       2024-07-23

## Update Post Data <a class="anchor" id="chapter4"></a>

In [27]:
# Output dailt post data
legacy_post = pd.read_csv(output_filepath+'SMSposts.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_post.to_csv(input_filepath+'SMSposts_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_post = pd.read_csv(input_filepath+'SMSPosts.csv',index_col=False)
updated_post['Pub Id'] = updated_post['Pub Id'].replace({461850.0: 461896.0})
updated_post['Pub Id'] = updated_post['Pub Id'].replace({461949.0: 461916.0})
pattern = 'SMS|SaltMedia'
drop = updated_post['Pub Name'].str.contains(pattern, case=False, na=False)
updated_post = updated_post.loc[drop]
today = date.today()
first_date_of_month = today.replace(day=1)
updated_post = updated_post.loc[updated_post['Pub Id'].isna() == False]
current_year = datetime.now().year
updated_post['Dates'] = pd.to_datetime(updated_post['Dates'] + f'/{current_year}', format="%m/%d/%Y")#.apply(lambda x: x.replace(year=current_year))
updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
updated_post = updated_post[updated_post['Dates']<=yesterday]
#updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates'] )
min_date = updated_post['Dates'].min()
legacy_post = legacy_post[legacy_post['Dates']<min_date]
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates']).dt.strftime('%m/%d/%Y')
post = [legacy_post, updated_post]
output_post= pd.concat(post, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_post)
output_post['Pub Id'] = output_post['Pub Id'].astype(str)
output_post['Pub Name'] = output_post['Pub Name'].astype(str)
output_post = output_post.loc[output_post['Pub Name'].str.contains(pattern, case = False, na = False)]
output_post['Dates'] =  pd.to_datetime(output_post['Dates']) 
output_post.to_csv(output_filepath+'SMSposts.csv', index = False)

            Dates    Pub Id                           Pub Name  Raw Posts  \
0      01/01/2022  461227.0       LM_SMS_Edemographic_247Loans          0   
1      01/01/2022  461313.0  LM_SMS_NorthPointDirect_RentToOwn          0   
2      01/01/2022  460921.0            LM_SMS_Spark_CreditCard      17102   
3      01/01/2022  461147.0                 461147_ARM.SYF_SMS      15884   
4      01/01/2022  460917.0                   460917_JM.OP_SMS       9999   
...           ...       ...                                ...        ...   
41797  07/28/2024  461867.0                   461867_I.TFE_SMS          0   
41798  07/28/2024  461039.0                 461039_DOT.818_SMS          0   
41799  07/28/2024  461843.0          LM_SMS_Spark_Sweepstakes2          0   
41800  07/28/2024  461842.0           LM_SMS_Spark_CreditCard2          0   
41801  07/28/2024  461348.0                   461348_I.CHQ_SMS          0   

       GeoIP Good  % IP Good  GeoIP Bad  % IP Bad  Goodhyg  % Good  ...  \


## Update Optout <a class="anchor" id="chapter5"></a>

In [28]:
from os import listdir
from os.path import isfile, join
import datetime as dt
                                                                     
smsreport_files = [f for f in listdir(smsreport_filepath) if isfile(join(smsreport_filepath, f)) & ('SMS_SC_SS_Report' in f)]
""" 
most_recent_smsreport_date = max([dt.datetime.strptime((date.split('.csv')[0][-8:]), "%Y%m%d") for date in smsreport_files])
smsreport_year = most_recent_smsreport_date.year
smsreport_month = most_recent_smsreport_date.month
segement_day = most_recent_smsreport_date.day

most_recent_smsreport_file = "SMS_SC_SS_Report{0}{1:0>2}{2:0>2}.csv".format(smsreport_year,smsreport_month,segement_day)

smsreport_report = pd.read_csv( smsreport_filepath+ most_recent_smsreport_file)

smsreport_report['Date'] = pd.to_datetime(most_recent_smsreport_date)
smsreport_report['Shortcode Name'] = ''
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('FLC'),'Shortcode Name' ] ='FLC'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('HZB'),'Shortcode Name' ] ='HZB'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('MBC'),'Shortcode Name' ] ='MBC'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('CSS'),'Shortcode Name' ] ='CSS'
legacy_smsreport = pd.read_csv(output_filepath+'SMS_SC_SS_Report.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_smsreport.to_csv(input_filepath+'SMS_SC_SS_Report_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
legacy_smsreport['Date'] = pd.to_datetime(legacy_smsreport['Date'])
min_date = smsreport_report['Date'].min()
legacy_smsreport = legacy_smsreport[legacy_smsreport['Date']<min_date]
full_smsreport = [legacy_smsreport, smsreport_report]
output_smsreport = pd.concat(full_smsreport, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_smsreport)
output_smsreport.to_csv(output_filepath+'SMS_SC_SS_Report.csv', index = False)
"""
df = pd.DataFrame()
for i in smsreport_files:
    smsreport_report = pd.read_csv(smsreport_filepath+i)
    most_recent_smsreport_date = dt.datetime.strptime((i.split('.csv')[0][-8:]), "%Y%m%d")
    smsreport_report['Date'] = pd.to_datetime(most_recent_smsreport_date)
    smsreport_report['Shortcode Name'] = ''
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('Dollar Sensei'),'Shortcode Name' ] ='DSS'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('FLC'),'Shortcode Name' ] ='FLC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('HZB'),'Shortcode Name' ] ='HZB'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('MBC'),'Shortcode Name' ] ='MBC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('CSS'),'Shortcode Name' ] ='CSS'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('SVT'),'Shortcode Name' ] ='SVT'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('UAA'),'Shortcode Name' ] ='UAA'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('PRC'),'Shortcode Name' ] ='PRC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('UAATF'),'Shortcode Name' ] ='UAATF'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('Aid4Famlilies'),'Shortcode Name' ] ='A4FLC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18332686782'),'Shortcode Name' ] ='MFATF'
    #added
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18332685688'),'Shortcode Name' ] ='N3GTF'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18338987652'),'Shortcode Name' ] ='FBHTF'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18333275117'),'Shortcode Name' ] ='FRHTF'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18449395079'),'Shortcode Name' ] ='TTDTF'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18888656812'),'Shortcode Name' ] ='DATF'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18449674656'),'Shortcode Name' ] ='TRSTF'
    df = pd.concat([df,smsreport_report])
df.to_csv(output_filepath+'SMS_SC_SS_Report.csv', index = False)


# update apikey <a class="anchor" id="chapter6"></a>

In [29]:
from os import listdir
from os.path import isfile, join
import datetime as dt
                                                                     
smsreport_files = [f for f in listdir(smsapikey_filepath) if isfile(join(smsapikey_filepath, f)) & ('apikeyreport' in f)]

df = pd.DataFrame()
for i in smsreport_files:
    smsapikey_report = pd.read_csv(smsapikey_filepath+i)
    smsapikey_report['Date'] = pd.to_datetime(smsapikey_report['Date'])
    smsapikey_report['DP.SV'] = smsapikey_report['Apikey'].str.split(" |-",expand = True)[0]
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.FD',na= False),'DP.SV' ] ='I.FD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.RTOD',na= False),'DP.SV' ] ='I.RTOD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.LD',na= False),'DP.SV' ] ='I.LD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.NFD',na= False),'DP.SV' ] ='I.NFD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('A4F',na= False),'DP.SV' ] ='I.A4F'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('MFA',na= False),'DP.SV' ] ='I.MFA'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('N3G',na= False),'DP.SV' ] ='I.N3G'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('FBH',na= False),'DP.SV' ] ='I.FBH'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('THM',na= False),'DP.SV' ] ='I.THM'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('FRH',na= False),'DP.SV' ] ='I.FRH'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('DA',na= False),'DP.SV' ] ='I.DA'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('TRS',na= False),'DP.SV' ] ='PJ.B'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('TTD',na= False),'DP.SV' ] ='TLG.PL.T'
    df = pd.concat([df,smsapikey_report])
df = df.drop_duplicates()
df.to_csv(output_filepath+'SMS_SC_SS_Apikey.csv', index = False)


## Update Flow <a class="anchor" id="chapter7"></a>

In [30]:
legacy_flows = pd.read_csv(output_filepath+'SMS_SC_SS_Flows.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_flows.to_csv(input_filepath+'SMS_SC_SS_Flows_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_flows = pd.read_csv(input_filepath+'SMS_SC_SS_Flows.csv',index_col=False)
legacy_flows['Period'] = pd.to_datetime(legacy_flows['Period'],format='mixed')
min_date = updated_flows['Period'].min()
legacy_flows = legacy_flows[legacy_flows['Period']<min_date]
flows = [legacy_flows, updated_flows]
output_flows = pd.concat(flows, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
# print(output_flows)
output_flows.to_csv(output_filepath+'SMS_SC_SS_Flows.csv', index = False)


## Update Creatives Data <a class="anchor" id="chapter8"></a>

In [31]:
legacy_creatives = pd.read_csv(output_filepath+'SMS_SC_SS_CreativesStats.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_creatives.to_csv(input_filepath+'SMS_SC_SS_CreativesStats_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_creatives = pd.read_csv(input_filepath+'creativestats.csv',index_col=False)
legacy_creatives['Tstamp'] = pd.to_datetime(legacy_creatives['Tstamp'],format='mixed')
min_date = updated_creatives['Tstamp'].min()
legacy_creatives = legacy_creatives[legacy_creatives['Tstamp']<min_date]
creativestats = [legacy_creatives, updated_creatives]
output_creatives = pd.concat(creativestats, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
# print(output_creatives)
output_creatives['Tstamp'] = pd.to_datetime(output_creatives['Tstamp'],format='mixed')
output_creatives.to_csv(output_filepath+'SMS_SC_SS_CreativesStats.csv', index = False)


In [32]:
# output_creatives['Tstamp'].astype(str).str[:10].unique()

## Update Jobs <a class="anchor" id="chapter9"></a>

In [33]:
legacy_jobs = pd.read_csv(output_filepath+'SMS_SC_SS_Jobs.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_jobs.to_csv(input_filepath+'SMS_SC_SS_Jobs_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
updated_jobs = pd.read_csv(input_filepath+'jobs.csv',index_col=False)
legacy_jobs['scheduled_tstamp'] = pd.to_datetime(legacy_jobs['scheduled_tstamp'],format = "mixed")
min_date = updated_jobs['scheduled_tstamp'].min()
legacy_jobs = legacy_jobs[legacy_jobs['scheduled_tstamp']<min_date]
jobs = [legacy_jobs, updated_jobs]
output_jobs = pd.concat(jobs, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_jobs)
output_jobs.to_csv(output_filepath+'SMS_SC_SS_Jobs.csv', index = False)

           id                                        name  \
0      237891                       B2USRA_1 [2021-10-01]   
1      238383                       B2USRA_1 [2021-10-02]   
2      238832                       B2USRA_1 [2021-10-03]   
3      239249                       B2USRA_1 [2021-10-04]   
4      239695                       B2USRA_1 [2021-10-05]   
...       ...                                         ...   
41414  690691    SS_HZB_AL-PL-2-30DC-VZN_13476_JT_28Jul24   
41415  690695  SS_HZB_AL-PL-2-30DC-VZN_13476v1_JT_28Jul24   
41416  690698       SS_DSS_GR-PL-30DC-TMO_13362_P_28Jul24   
41417  690700           SS_FLC_GR-PL-30DC_13399_P_28Jul24   
41418  690702    SS_DSS_JET-ZTA-1PLD-21DC_12945_P_28Jul24   

                                   offer  shortcode             start_tstamp  \
0      B2USRA - Cheris Studio Benefits A      70610  2021-10-01 13:00:09 EST   
1      B2USRA - Cheris Studio Benefits A      70610  2021-10-02 13:00:10 EST   
2      B2USRA - Cheris Stud

In [34]:
updated_jobs

,id,name,offer,shortcode,start_tstamp,end_tstamp,scheduled_tstamp,status_text,remote_status,segments,...,sent,delivered,not_delivered,optout,clicks,conversions,revenue,cost,ecpm,roi
0,689824,SS_MBC_NPD-RTO-7DC_13362_P_26Jul24,13362 - MBC,80837,2024-07-26 11:31:21 EST,2024-07-26 11:34:04 EST,2024-07-26T15:30:59Z,Sending Complete,NaN,MBC_NPD.RTO_7DC,...,1183,1182,5,28,332,0,0.0,1.4196,0.0,-1.4196
1,689825,SS_MBC_I-RC-15DC_12972_P_26Jul24,12972 - MBC,80837,2024-07-26 11:30:57 EST,2024-07-26 11:38:41 EST,2024-07-26T15:30:18Z,Sending Complete,NaN,MBC_I.RC_15DC,...,5800,5798,21,55,975,0,0.0,6.9600,0.0,-6.9600
2,689826,SS_MBC_I-RC-15DC_13661_PT_26Jul24,13661 - MBC,80837,2024-07-26 13:03:59 EST,2024-07-26 13:27:00 EST,2024-07-26T17:00:39Z,Sending Complete,NaN,MBC_I.RC_15DC,...,16,16,1983,0,0,0,0.0,0.0192,0.0,-0.0192
3,689827,SS_FLC_I-RC-30DC-VZN-MAY24_12978_P_26Jul24,12978 - FLC,51797,2024-07-26 13:02:58 EST,2024-07-26 13:14:14 EST,2024-07-26T17:00:14Z,Sending Complete,NaN,FLC_I.RC_30DC_VZN_MAY24,...,800,800,16,0,107,0,0.0,0.9600,0.0,-0.9600
4,689830,SS_HZB_SM-SRV-30DC-VZN_13498_P_26Jul24,13498 - HZB,44345,2024-07-26 13:02:58 EST,2024-07-26 13:39:41 EST,2024-07-26T17:00:11Z,Sending Complete,NaN,HZB_SM.SRV_30DC_VZN,...,4631,4544,251,52,419,0,0.0,32.4240,0.0,-32.4240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,690691,SS_HZB_AL-PL-2-30DC-VZN_13476_JT_28Jul24,13476 - HZB,44345,2024-07-28 15:02:16 EST,2024-07-28 15:06:34 EST,2024-07-28T19:00:48Z,Sending Complete,NaN,HZB_AL.PL.2_30DC_VZN,...,1376,1332,55,14,95,0,0.0,9.6320,0.0,-9.6320
181,690695,SS_HZB_AL-PL-2-30DC-VZN_13476v1_JT_28Jul24,13476v1 - HZB,44345,2024-07-28 15:00:46 EST,2024-07-28 15:06:17 EST,2024-07-28T19:00:07Z,Sending Complete,NaN,HZB_AL.PL.2_30DC_VZN,...,1472,1420,61,6,102,0,0.0,10.3040,0.0,-10.3040
182,690698,SS_DSS_GR-PL-30DC-TMO_13362_P_28Jul24,13362 - DSS,31232,2024-07-28 15:00:32 EST,2024-07-28 15:06:38 EST,2024-07-28T19:00:15Z,Sending Complete,NaN,DSS_GR.PL_30DC_TMO,...,1476,1397,26,6,168,0,0.0,1.7712,0.0,-1.7712
183,690700,SS_FLC_GR-PL-30DC_13399_P_28Jul24,13399 - FLC,51797,2024-07-28 15:00:38 EST,2024-07-28 15:06:31 EST,2024-07-28T19:00:23Z,Sending Complete,NaN,FLC_GR.PL_30DC,...,1797,1796,51,12,82,0,0.0,2.1564,0.0,-2.1564


In [35]:
output_jobs['start_tstamp'].str[:10].unique()

array(['2021-10-01', '2021-10-02', '2021-10-03', ..., '2024-07-26',
       '2024-07-27', '2024-07-28'], dtype=object)

## Update Offers <a class="anchor" id="chapter10"></a>

In [36]:
offers_old = pd.read_csv(output_filepath + 'SMS_SC_SS_Offers.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong
offers_old.to_csv(input_filepath+'SMS_SC_SS_Offers_backup.csv', index = False)
# This is the offer raw data downloaded by the outsource 
offers = pd.read_csv(input_filepath+'SMS_SC_SS_Offers.csv',index_col=False)
offers['hitpath_offer_id'] = offers['name'].str.split(' ',expand = True)[0].str.strip()
offers = offers[['id', 'name','hitpath_offer_id', 'type', 'status', 'redirect_type', 'conversion_event',
       'conversion_payout', 'currency']]
print(offers.head(5))
offers.to_csv(output_filepath+'SMS_SC_SS_Offers.csv', index = False)

      id                name hitpath_offer_id type  status redirect_type  \
0  20267   6963 - WM UBG CSS             6963  sms  active          HTTP   
1  40816   12088 - BE OE HZB            12088  sms  active          HTTP   
2  48116  12610 - AVL CL SVT            12610  sms  active          HTTP   
3  48117  12611 - AVL QL HZB            12611  sms  active          HTTP   
4  40935   12163 - BK OD HZB            12163  sms  active          HTTP   

  conversion_event  conversion_payout currency  
0              CPA                  0      USD  
1              CPA                  0      USD  
2              CPA                  0      USD  
3              CPA                  0      USD  
4              CPA                  0      USD  


In [37]:
end_time = time.time()

In [38]:
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

Execution time: 2192.63 seconds
